In [20]:
import os

import dill
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cvxpy as cp
import math

from ecoli.library.lattice_utils import count_to_concentration

%matplotlib inline

os.chdir(os.path.expanduser('~/dev/vEcoli/'))

from ecoli.processes.metabolism_redux_classic import NetworkFlowModel, FlowResult
from ecoli.processes.registries import topology_registry
TOPOLOGY = topology_registry.access("ecoli-metabolism-redux")

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# load model
time = '2000'
date = '2025-09-10'
experiment = 'GLOP_sum_pos_diversity'
condition = 'basal'
entry = f'{experiment}_{time}_{date}'
folder = f'out/flow-partition/{condition}/{entry}/'

output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

metabolism = agent['ecoli-metabolism-redux-classic']

In [3]:
# load original model
time = '2000'
date = '2025-09-09'
experiment = 'GLOP'
condition = 'basal'
entry = f'{experiment}_{time}_{date}'
folder = f'out/flow-partition/{condition}/{entry}/'

output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output_original = output['agents']['0']
fba_original = output_original['listeners']['fba_results']
mass_original = output_original['listeners']['mass']
bulk_original = pd.DataFrame(output_original['bulk'])
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

metabolism_original = agent['ecoli-metabolism-redux-classic']

# Health Comparison Plots

In [4]:
plot_items = ['dry_mass', 'volume', 'cell_mass', 'mRna_mass', 'rna_mass', 'growth']

# optional time vectors per key (leave as empty dicts if not using)
try:
    t_new
except NameError:
    t_new = {}
try:
    t_old
except NameError:
    t_old = {}


rows = 2
ncols = math.ceil(len(plot_items) / rows)
fig = make_subplots(rows=rows, cols=ncols, subplot_titles=plot_items)

for idx, key in enumerate(plot_items):
    r, c = idx // ncols + 1, idx % ncols + 1

    y_new = np.asarray(mass.get(key, []))
    y_old = np.asarray(mass_original.get(key, []))

    # separate x for each series so different lengths are fine
    x_new = np.asarray(t_new[key]) if key in t_new else np.arange(len(y_new))
    x_old = np.asarray(t_old[key]) if key in t_old else np.arange(len(y_old))

    if y_new.size:
        fig.add_trace(
            go.Scatter(
                x=x_new, y=y_new, mode="lines",
                name="Updated FBA", legendgroup="new",
                showlegend=(idx == 0),
                line=dict(color="#1f77b4")
            ),
            row=r, col=c
        )
    if y_old.size:
        fig.add_trace(
            go.Scatter(
                x=x_old, y=y_old, mode="lines",
                name="Original FBA", legendgroup="old",
                line=dict(dash="dash", color="darksalmon"),
                showlegend=(idx == 0)
            ),
            row=r, col=c
        )

    fig.update_xaxes(title_text="Time (s)", row=r, col=c)
    fig.update_yaxes(title_text=key, row=r, col=c)

fig.update_layout(height=500, width=800*rows, title="Updated vs Original FBA", template="plotly_white", paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1')
fig.show()
# fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/WC_health_comparison.png", scale=3)

In [5]:
print(ncols)
fig.update_layout(height=400, width=600*ncols, title="Updated vs Original FBA", template="plotly_white", paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1')
fig.show()
# fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/WC_health_comparison.png", scale=3, width=1000, height=600)

3


# kinetic scatterplot

In [6]:
# from ecoli.library.schema import concentrationToCounts
metabolism.concentrationToCounts(1E-3)

np.int64(983)

In [7]:
target_kinetic_fluxes = pd.DataFrame(fba["target_kinetic_fluxes"], columns=metabolism.kinetic_constraint_reactions).mean(axis=0)
simulated_fluxes = pd.DataFrame(fba["estimated_fluxes"], columns=metabolism.reaction_names).mean(axis=0)
simulated_kinetic_fluxes = simulated_fluxes[metabolism.kinetic_constraint_reactions]

target_kinetic_fluxes = np.log10(target_kinetic_fluxes + 1e-6)
simulated_kinetic_fluxes = np.log10(simulated_kinetic_fluxes + 1e-6)

# plot scatter plot
fig = px.scatter(x=target_kinetic_fluxes, y=simulated_kinetic_fluxes,
                 labels={'x': 'log(Target kinetic fluxes)', 'y': 'log(Simulated kinetic fluxes)'},
                 title='Kinetic fluxes: Target vs Simulated')
fig.add_shape(type='line', x0=-6, y0=-6, x1=target_kinetic_fluxes.max().max(), y1=target_kinetic_fluxes.max().max(),
              line=dict(color='Red', dash='dash'))

fig.update_layout(showlegend=False,
                  template='plotly_white',
                  paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
                  xaxis=dict(gridcolor='lightgray'),
                  yaxis=dict(gridcolor='lightgray')
                  )
fig.show()
fig.write_image('/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/WC_kinetic_w_diversity.png', width = 800, height = 600, scale=3)

In [8]:

simulated_kinetic_fluxes

1.1.1.39-RXN                                                                                                              -6.000000
1.1.1.83-RXN                                                                                                              -6.000000
1.13.11.16-RXN                                                                                                            -6.000000
2.1.1.79-RXN-S-ADENOSYLMETHIONINE/CPD-18361//CPD-18373/ADENOSYL-HOMO-CYS/PROTON.67.                                       -6.000000
2.1.1.79-RXN-S-ADENOSYLMETHIONINE/CPD-18362//CPD-18406/ADENOSYL-HOMO-CYS/PROTON.67.                                       -6.000000
                                                                                                                             ...   
UGD-RXN                                                                                                                    2.016868
UNDECAPRENYL-DIPHOSPHATASE-RXN[CCO-CYTOSOL]-UNDECAPRENYL-DIPHOSPHATE/WATER//

In [9]:
target_kinetic_fluxes

1.1.1.39-RXN                                                                                                               4.291378
1.1.1.83-RXN                                                                                                               3.359361
1.13.11.16-RXN                                                                                                             1.518514
2.1.1.79-RXN-S-ADENOSYLMETHIONINE/CPD-18361//CPD-18373/ADENOSYL-HOMO-CYS/PROTON.67.                                        2.268901
2.1.1.79-RXN-S-ADENOSYLMETHIONINE/CPD-18362//CPD-18406/ADENOSYL-HOMO-CYS/PROTON.67.                                        2.268901
                                                                                                                             ...   
UGD-RXN                                                                                                                    1.892095
UNDECAPRENYL-DIPHOSPHATASE-RXN[CCO-CYTOSOL]-UNDECAPRENYL-DIPHOSPHATE/WATER//

In [10]:
target_kinetic_fluxes.mean(axis=0)

np.float64(2.679921861487223)

In [11]:
target_kinetic_fluxes = pd.DataFrame(fba_original["target_kinetic_fluxes"], columns=metabolism_original.kinetic_constraint_reactions)
simulated_fluxes = pd.DataFrame(fba_original["estimated_fluxes"], columns=metabolism_original.reaction_names)
simulated_kinetic_fluxes = simulated_fluxes[metabolism_original.kinetic_constraint_reactions]

target_kinetic_fluxes = np.log10(target_kinetic_fluxes + 1e-6)
simulated_kinetic_fluxes = np.log10(simulated_kinetic_fluxes + 1e-6)

# plot scatter plot
fig = px.scatter(x=target_kinetic_fluxes.mean(axis=0), y=simulated_kinetic_fluxes.mean(axis=0),
                 labels={'x': 'log(Target kinetic fluxes)', 'y': 'log(Simulated kinetic fluxes)'},
                 title='Kinetic fluxes: Target vs Simulated')
fig.add_shape(type='line', x0=-6, y0=-6, x1=target_kinetic_fluxes.max().max(), y1=target_kinetic_fluxes.max().max(),
              line=dict(color='Red', dash='dash'))
fig.update_layout(showlegend=False,
                  template='plotly_white',
                  paper_bgcolor='#f6f4f1', plot_bgcolor='#f6f4f1',
                  xaxis=dict(gridcolor='lightgray'),
                  yaxis=dict(gridcolor='lightgray')
                  )
fig.show()
fig.write_image('/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/flow_partition/WC_kinetic_wo_diversity.png', width = 800, height = 600, scale=3)

# See if flow_partitioning is working for some reactions of interest

In [12]:
# get commonly stored variables
metabolism = agent['ecoli-metabolism-redux-classic']
stoichiometry = metabolism.stoichiometry.copy()
reaction_names = metabolism.reaction_names
fba_new_reaction_ids = metabolism.parameters["fba_new_reaction_ids"]
fba_reaction_ids_to_base_reaction_ids = metabolism.parameters['fba_reaction_ids_to_base_reaction_ids']
metabolites = metabolism.metabolite_names.copy()
binary_kinetic_idx = metabolism.binary_kinetic_idx
exchange_molecules = metabolism.exchange_molecules

S = stoichiometry .copy()
S = pd.DataFrame(S, index=metabolites , columns=reaction_names )
homeostatic_count = pd.DataFrame(fba["homeostatic_metabolite_counts"], columns=metabolism.homeostatic_metabolites).loc[24, :]
homeostatic = pd.DataFrame(fba["target_homeostatic_dmdt"], columns=metabolism.homeostatic_metabolites).loc[24, :]
maintenance = pd.DataFrame(fba["maintenance_target"][1:], columns=['maintenance_reaction']).iat[24, 0]
kinetic = pd.DataFrame(fba["target_kinetic_fluxes"], columns=metabolism.kinetic_constraint_reactions).loc[24, :].copy()
kinetic_reaction_ids = metabolism.kinetic_constraint_reactions.copy()

allowed_exchange_uptake = metabolism.allowed_exchange_uptake
FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]

In [61]:
def get_subset_S(S, met_of_interest):
    S_met = S.loc[met_of_interest, :]
    S_met = S_met.loc[:,~np.all(S_met == 0, axis=0)]
    return S_met, S_met.columns

def get_keys(dict, value):
    return [key for key in dict if dict[key] == value]

def test_NetworkFlowModel(objective_weights,
                          uptake_addition = set([]), uptake_removal = set([]), new_exchange_molecules = set([]),
                          add_metabolite = None, add_reaction = None, add_kinetic = None, remove_reaction = None, force_reaction = None,
                          add_homeostatic_demand = None, solver_choice=cp.GLOP):
    # update exchanges
    uptake = metabolism.allowed_exchange_uptake.copy()
    uptake = set(uptake)
    uptake = uptake | uptake_addition
    uptake = uptake - uptake_removal

    exchange_molecules = metabolism.exchange_molecules.copy()
    exchange_molecules = exchange_molecules | new_exchange_molecules

    # update stoichiometry
    reaction_names = metabolism.reaction_names.copy()
    kinetic_reaction_ids = metabolism.kinetic_constraint_reactions.copy()
    kinetic = pd.DataFrame(fba["target_kinetic_fluxes"], columns=metabolism.kinetic_constraint_reactions).loc[24, :].copy()
    metabolites = metabolism.metabolite_names.copy()
    homeostatic_counts = homeostatic_count.copy() * metabolism.counts_to_molar.asNumber()

    S_new = stoichiometry.copy()

    if add_metabolite is not None: # add to metabolites list because they are currently not included in the model
        for m in add_metabolite:
            if m not in metabolites:
                metabolites.append(m)
        # append rows of zeros to S_new of length add_metabolite
        S_new = np.concatenate((S_new, np.zeros((len(add_metabolite), S_new.shape[1]))), axis=0)

    if add_reaction is not None:
        # assert add_reaction is a dictionary
        assert isinstance(add_reaction, dict)

        for r,s in add_reaction.items():
            if r not in reaction_names:
                reaction_names.append(r)
            # append columns of reaction stoich to S_new of length add_reaction
            new_reaction = np.zeros((S_new.shape[0], 1))
            for m, v in s.items():
                new_reaction[metabolites.index(m), 0] = v
            S_new = np.concatenate((S_new, new_reaction), axis=1)

    if add_kinetic is not None:
        # assert add_kinetic is a dictionary
        assert isinstance(add_kinetic, dict)

        for r, v in add_kinetic.items():
            if r not in kinetic_reaction_ids:
                kinetic_reaction_ids.append(r)
                kinetic[r] = v

    if remove_reaction is not None:
        for r in remove_reaction:
            r_idx = reaction_names.index(r)
            S_new = np.delete(S_new, r_idx, axis=1)
            reaction_names.remove(r)
            if r in kinetic_reaction_ids:
                kinetic_reaction_ids.remove(r)
                del kinetic[r]

    if force_reaction is not None:
        force_reaction_idx = np.array([reaction_names.index(r) for r in force_reaction])
    else:
        force_reaction_idx = force_reaction

    if add_homeostatic_demand is not None:
        # assert add_homeostatic_demand is a set
        assert isinstance(add_homeostatic_demand, list)

        for met in add_homeostatic_demand:
            homeostatic[met] = 100
            homeostatic_counts[met] = 1

    # Solve NetworkFlowModel
    model = NetworkFlowModel(
            stoich_arr=S_new,
            metabolites=metabolites,
            reactions=reaction_names,
            homeostatic_metabolites=metabolism.homeostatic_metabolites,
            kinetic_reactions=kinetic_reaction_ids,
            free_reactions=FREE_RXNS)
    model.set_up_exchanges(exchanges=exchange_molecules, uptakes=uptake)
    solution: FlowResult = model.solve(
            homeostatic_concs=homeostatic_counts, # in conc
            homeostatic_dm_targets=np.array(list(dict(homeostatic).values())), # *10^7
            maintenance_target=maintenance, # *10^6 ish
            kinetic_targets=np.array(list(dict(kinetic).values())), # *10^6 ish
            # binary_kinetic_idx=binary_kinetic_idx, #7646
            binary_kinetic_idx=None,
            force_flow_idx=force_reaction_idx,
            objective_weights=objective_weights, #same
            upper_flux_bound= 100000000, # increase to 10^9 or 8 because notebook runs FlowResult using Counts, WC runs using conc.
            solver=solver_choice) #SCS. ECOS, MOSEK
    return solution.objective, solution.velocities, reaction_names, S_new, metabolites, kinetic

In [84]:
objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.000001, 'diversity': 1E-2}
add_reaction={
    'TEMP-TRANS-TRIMETHYLAMINE[p] -> [c]': {
        'TRIMETHYLAMINE[c]' : 1,
        'TRIMETHYLAMINE[p]' : -1,
    },
    'TEMP-TRANS-TRIMETHYLAMINE[p] -> [c] (REVERSE)': {
        'TRIMETHYLAMINE[c]' : -1,
        'TRIMETHYLAMINE[p]' : 1,
    },
    'TEMP-GAMMA-BUTYROBETAINYL-COA-DEGRADATION': {
        'GAMMA-BUTYROBETAINYL-COA[c]' : -1,
        'CROTONYL-COA[c]' : 1,
        'TRIMETHYLAMINE[c]': 1,
    },
}
new_exchange_molecules = set(['TRIMETHYLAMINE[p]'])
uptake_addition = set(['CARNITINE[e]'])
uptake_removal = set(['GLC[p]'])
force_reaction = ['RXN0-3601']

In [86]:
oofv, solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights,
                                            new_exchange_molecules= new_exchange_molecules,
                                            uptake_addition=uptake_addition,
                                            uptake_removal=uptake_removal,
                                            add_reaction=add_reaction,
                                            # force_reaction=force_reaction

)
oofv

np.float64(39237.87036550263)

In [87]:
sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids
        else 'TEMP' if id in add_reaction.keys()
        else 'Old Reactions'
            for id in test_reaction_names
    ]
}, index=test_reaction_names)


In [88]:
met_of_interest = ['CARNITINE[c]', 'GAMMA-BUTYROBETAINYL-COA[c]', 'TRIMETHYLAMINE[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
CROBETREDUCT-RXN (reverse),738771.238636,New Reactions,False
LCARNCOALIG-RXN,738571.238636,New Reactions,False
RXN-18258-GAMMA-BUTYROBETAINE/NADH/OXYGEN-MOLECULE/PROTON//SUCC-S-ALD/TRIMETHYLAMINE/NAD/WATER.85.,100.000000,Old Reactions,False
RXN-18258-GAMMA-BUTYROBETAINE/NADPH/OXYGEN-MOLECULE/PROTON//SUCC-S-ALD/TRIMETHYLAMINE/NADP/WATER.87.,100.000000,Old Reactions,False
RXN-18376-CHOLINE/NADH/OXYGEN-MOLECULE/PROTON//GLYCOLALDEHYDE/TRIMETHYLAMINE/NAD/WATER.77.,-0.000000,Old Reactions,False
RXN-18376-CHOLINE/NADPH/OXYGEN-MOLECULE/PROTON//GLYCOLALDEHYDE/TRIMETHYLAMINE/NADP/WATER.79.,-0.000000,Old Reactions,False
RXN-5921-CARNITINE/NADH/OXYGEN-MOLECULE/PROTON//CPD-16618/TRIMETHYLAMINE/NAD/WATER.74.,-0.000000,Old Reactions,False
RXN-5921-CARNITINE/NADPH/OXYGEN-MOLECULE/PROTON//CPD-16618/TRIMETHYLAMINE/NADP/WATER.76.,-0.000000,Old Reactions,False
RXN-8638-CARNITINE/ATP/WATER//CARNITINE/ADP/Pi/PROTON.45.,738671.238636,Old Reactions,False
RXN-8638-GAMMA-BUTYROBETAINYL-COA/ATP/WATER//GAMMA-BUTYROBETAINYL-COA/ADP/Pi/PROTON.75.,-0.000000,Old Reactions,False


In [20]:
pd.DataFrame(fba["estimated_fluxes"], columns=metabolism.reaction_names).mean(axis=0)

1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN                                                                                     0.000
1.1.1.127-RXN                                                                                                         778.108
1.1.1.127-RXN (reverse)                                                                                               778.108
1.1.1.215-RXN (reverse)                                                                                                 0.000
1.1.1.251-RXN                                                                                                         778.108
                                                                                                                     ...     
YIAE1-RXN (reverse)                                                                                                     0.000
glycogen-monomer-extension                                                                                          12